# Medallion Architecture: Healthcare Example with Pandas

## 👩‍💼 **Client Scenario:**
You are a data engineer working with a healthcare provider. Your client has access to synthetic patient data generated from Synthea, which simulates real-world medical records. The client needs help building a structured pipeline to analyze patient and encounter data in a way that supports business intelligence and decision-making.

Your task is to build a medallion architecture pipeline with the following objectives:
 
1. Load and inspect raw patient, encounter, and medication data.
2. Clean and prepare the data into consistent, analysis-ready tables.
3. Create enriched views by joining datasets to gain a full picture of the patient journey.
4. Calculate and save key performance indicators (KPIs).
5. Generate visualizations to support reporting.

# 🥉 BRONZE LAYER

The raw data landing zone. It contains unprocessed datasets as extracted from the source. No cleaning or transformations are done here, just data dumps.

We'll use real-world-style healthcare datasets generated from Synthea: patients, encounters, and medications.


In [ ]:
%pip install matplotlib 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Create folder structure
os.makedirs("bronze", exist_ok=True)
os.makedirs("silver", exist_ok=True)
os.makedirs("gold", exist_ok=True)

### 🧩 **Step 1 – Your Task as the Data Engineer:**
The client has given you access to raw healthcare datasets. The job here is simple:
 
- Load the CSV files from the Synthea data dump.
- Explore them to understand what kind of information you're working with.
- Do not modify the data yet — just inspect and get familiar with it.

In [ ]:
patients = pd.read_csv("bronze/patients.csv")
encounters = pd.read_csv("bronze/encounters.csv")
medications = pd.read_csv("bronze/medications.csv")

After loading the data, ask yourself the following:
1. How many columns and rows are in each dataset?
2. Which columns look like IDs or date values?
3. Can you spot any missing values or strange formatting?

Explore and Inspect Raw Data (Optional (but recommended))

In [ ]:
print("Patients:")
print(patients.head())
print("\nEncounters:")
print(encounters.head())
print("\nMedications:")
print(medications.head())

# 🥈 SILVER LAYER
Cleaned and normalized data. 

This stage standardises formats, filters out invalid or missing data, and prepares it for analysis by applying business logic and transformations. It also includes enriched datasets formed by joining multiple sources.

### 🛠️ **Step 2 – Build Trustworthy Tables:**
Now it’s time to prepare the data for analysis. We'll clean each dataset separately:
 
- Rename inconsistent column names
- Drop null or malformed values
- Convert strings to proper data types (especially for dates)
 
Complete and run the cleaning functions. Then:
1. Print the shape and dtypes of each cleaned table.
2. Spot any patient entries that might still be problematic.
3. Check how many medications were dropped due to missing encounters.

In [ ]:
def clean_patients(df):
    df.columns = df.columns.str.lower()
    df = df.dropna(subset=['id'])
    df = df.rename(columns={'id': 'patient_id'})
    return df[['patient_id', 'birthdate', 'gender', 'race', 'ethnicity', 'marital', 'state', 'city']]

def clean_encounters(df):
    df.columns = df.columns.str.lower()
    df = df.dropna(subset=['id', 'patient'])
    df = df.rename(columns={'id': 'encounter_id', 'patient': 'patient_id'})
    df['encounter_start'] = pd.to_datetime(df['start'], errors='coerce')
    df['encounter_end'] = pd.to_datetime(df['stop'], errors='coerce')
    df['encounter_month'] = df['encounter_start'].dt.to_period('M')
    return df[['encounter_id', 'patient_id', 'encounter_start', 'encounter_end', 'encounterclass', 'reasoncode', 'encounter_month']]

def clean_medications(df):
    df.columns = df.columns.str.lower()
    df = df.dropna(subset=['encounter'])
    df = df.rename(columns={'encounter': 'encounter_id'})
    return df[['encounter_id', 'description', 'start', 'stop', 'reasoncode']]

In [ ]:
patients_clean = clean_patients(patients)
encounters_clean = clean_encounters(encounters)
medications_clean = clean_medications(medications)

Save cleaned data to the `silver/` folder.

In [ ]:
patients_clean.to_csv("silver/patients_clean.csv", index=False)
encounters_clean.to_csv("silver/encounters_clean.csv", index=False)
medications_clean.to_csv("silver/medications_clean.csv", index=False)

### 🔗 **Step 3 – Connect the Dots:**
You now have cleaned tables. Let’s enrich the data:
 
- Join patients with their encounters
- Join encounters with medication records
- Create derived metrics such as the patient's age at the time of the encounter
 
After enriching the data:
1. How many unique patients have medication records?
2. What’s the average age of patients during encounters?
3. Which encounter class (inpatient, outpatient, etc.) is most frequent?

Join the datasets and create enriched healthcare records.

First join encounters with patients

Then join with medications, protect column integrity

In [ ]:
encounters_patients = encounters_clean.merge(patients_clean, on='patient_id', how='left')
encounters_full = encounters_patients.merge(
    medications_clean,
    on='encounter_id',
    how='left',
    suffixes=('', '_med')
)
print(encounters_full.head())

Add age at time of encounter

In [ ]:
encounters_full['birthdate'] = pd.to_datetime(encounters_full['birthdate'], utc=True)
encounters_full['age_at_encounter'] = ((encounters_full['encounter_start'] - encounters_full['birthdate']).dt.days / 365.25).astype(int)

Save enriched dataset in silver/

In [ ]:
encounters_full.to_csv("silver/encounters_full.csv", index=False)

# 🥇 GOLD LAYER
The final presentation layer. 
Contains aggregated, curated data suitable for reporting and decision-making. Here we compute KPIs and create visualizations to derive insights.
## 📊 **Step 4 – Business Insights for Decision Makers:**
It’s time to summarize the data into useful metrics (KPIs) and charts that executives and analysts can use.
You’ll compute:

- How many times patients typically interact with care providers
- The top reasons people visit clinics
- Which age groups are prescribed the most medication
 
1. Identify which age group has the most prescriptions.
2. Can you interpret what the top 3 encounter reasons suggest about patient needs?
3. Create one additional visualization that could be helpful to your client (e.g. encounter type by gender).

Compute healthcare insights from enriched data.

### KPI 1: Average number of encounters per patient
This metric helps understand how often patients are interacting with the healthcare system.

A higher number may indicate a population with frequent care needs, while a lower number could
suggest healthier individuals or gaps in access to care.

Calculate the number of encounters grouped per patient and then compute the average across all patients.


In [ ]:
encounters_per_patient = encounters_clean.groupby('patient_id').size()
avg_encounters = encounters_per_patient.mean().round(2)
print("Average Encounters per Patient:", avg_encounters)

### KPI 2: Most common encounter reasons

This metric identifies the most frequently cited reasons for patient visits.
It helps healthcare providers understand common health issues or patterns in patient demand.

Knowing the most common reasons for encounters can support decisions about resource allocation and patient education.


In [ ]:
top_reasons = encounters_clean['reasoncode'].value_counts().head(5)
print("\nTop 5 Encounter Reasons:\n", top_reasons)

### KPI 3: Medication frequency by age group
This metric analyzes how often medications are prescribed across different age groups.
It offers insight into age-specific treatment patterns, highlighting where chronic conditions or high medication use may be concentrated.

Grouping by age allows healthcare professionals to tailor preventive care and monitor age-related trends in prescriptions.


In [ ]:
encounters_full['age_group'] = pd.cut(encounters_full['age_at_encounter'], bins=[0, 18, 35, 50, 65, 100], labels=['0-18', '19-35', '36-50', '51-65', '66+'])
med_by_age = encounters_full.groupby('age_group')['description'].count()

### Save KPIs to CSV in gold/

In [ ]:
pd.DataFrame({'avg_encounters': [avg_encounters]}).to_csv("gold/avg_encounters.csv", index=False)
top_reasons.to_frame(name="count").to_csv("gold/top_reasons.csv")
med_by_age.to_frame(name="medication_count").to_csv("gold/medications_by_age.csv")

## 📊 Visualize a KPI (dealer's choice)

In [ ]:
med_by_age = pd.read_csv("gold/medications_by_age.csv", index_col=0)
med_by_age.plot(kind='bar', title='Medication Count by Age Group', figsize=(8,4))
plt.xlabel('Age Group')
plt.ylabel('Medication Count')
plt.tight_layout()
plt.savefig("gold/medication_by_age_plot.png")
plt.show()